__importnb__ imports notebooks as modules & packages.

[![Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/deathbeds/importnb/master?urlpath=lab/tree/readme.ipynb)[![Build Status](https://travis-ci.org/deathbeds/importnb.svg?branch=master)](https://travis-ci.org/deathbeds/importnb)[![PyPI version](https://badge.fury.io/py/importnb.svg)](https://badge.fury.io/py/importnb)![PyPI - Python Version](https://img.shields.io/pypi/pyversions/importnb.svg)![PyPI - Format](https://img.shields.io/pypi/format/importnb.svg)![PyPI - Format](https://img.shields.io/pypi/l/importnb.svg)[
![Conda](https://img.shields.io/conda/pn/conda-forge/importnb.svg)](https://anaconda.org/conda-forge/importnb)[
![GitHub tag](https://img.shields.io/github/tag/deathbeds/importnb.svg)](https://github.com/deathbeds/importnb/tree/master/src/importnb)

    pip install importnb
    
---

    conda install -c conda-forge importnb

# `importnb` works in Python and IPython

Use the `Notebook` context manager.

    >>> from importnb import Notebook

### For brevity

[`importnb.loader`](src/notebooks/loader.ipynb) will find notebooks available anywhere along the [`sys.path`](https://docs.python.org/2/library/sys.html#sys.path).

In [18]:
    from importnb.usage import notebook

In [19]:
    with __import__('importnb').Notebook(): 
        import readme

#### or explicity 

In [20]:
    from importnb import Notebook
    with Notebook(): 
        import readme

In [21]:
    foo = 42
    import readme
    assert readme.foo is 42
    assert readme.__file__.endswith('.ipynb')

### Modules may be reloaded 

The context manager is required to `reload` a module.

In [22]:
    from importlib import reload
    with Notebook():
        reload(readme)

### Partial loading

The [`importnb.loader.Notebook`](src/notebooks/loader.ipynb#Partial-Loader) will __import__ a notebook even if there is an exception by supplying the `exceptions` option.  The __exception__ is found on `module._exception`.

In [23]:
    with Notebook(exceptions=BaseException):
        try: from . import readme
        except: import readme

### Lazy imports

The `lazy` option will delay the evaluation of a module until one of its attributes are accessed the first time.

In [24]:
    with Notebook(lazy=True):
        import readme

## Capture Outputs

`importnb` can capture the `stdout`, `stderr`, and `display` in the context manager.  The arguments are similar to `IPython.util.capture.capture_output`.

In [25]:
    with Notebook(stdout=True, stderr=True, display=True) as output:
        import readme

### Docstring

The first markdown cell will become the module docstring.

In [26]:
    if __name__ == '__main__':
        print(readme.__doc__.splitlines()[0])

__importnb__ imports notebooks as modules & packages.


Meaning non-code blocks can be executeb by [doctest]().

In [29]:
    if __name__ == '__main__':
        __import__('doctest').testmod(readme)

# Import notebooks from files

Notebook names may not be valid Python paths.  In this case, use `Notebook.from_filename`.

       Notebook.from_filename('readme.ipynb')
       
Import under the `__main__` context.
       
       Notebook.from_filename('readme.ipynb', main=True)

# Parameterize Notebooks

Literal ast statements are converted to notebooks parameters.

In `readme`, `foo` is a parameter because it may be evaluated with ast.literal_val

In [30]:
    from importnb import Parameterize
    f = Parameterize().from_filename(readme.__file__)
    

The parameterized module is a callable that evaluates with different literal statements.

In [31]:
    assert callable(f)
    f.__signature__

<Signature (*, foo=42)>

    assert f().foo == 42
    assert f(foo='importnb').foo == 'importnb'

## Integrations


### IPython

#### [IPython Extension](src/notebooks/loader.ipynb#IPython-Extensions)

Avoid the use of the context manager using loading importnb as IPython extension.

    %load_ext importnb
    
`%unload_ext importnb` will unload the extension.

#### Default Extension

`importnb` may allow notebooks to import by default with 

    importnb-install
    
This extension will install a script into the default IPython profile startup that is called each time an IPython session is created.  

Uninstall the extension with `importnb-install`.

##### Run a notebook as a module

When the default extension is loaded any notebook can be run from the command line. After the `importnb` extension is created notebooks can be execute from the command line.

    ipython -m readme
    
> See the [deploy step in the travis build](https://github.com/deathbeds/importnb/blob/docs/.travis.yml#L19).

### py.test

`importnb` installs a pytest plugin when it is setup.  Any notebook obeying the py.test discovery conventions can be used in to pytest.  _This is great because notebooks are generally your first test._

### Setup

To package notebooks add `recursive-include package_name *.ipynb`

### [Watchdog](https://github.com/gorakhargosh/watchdog/tree/master/src/watchdog/tricks)

    pip install importnb[watch]

`importnb` exports a watchdog trick to watch files and apply command like operations on their module path.

#### Tricks File

For example, create a file called `tricks.yaml` containing

    tricks:
    - importnb.utils.watch.ModuleTrick:
          patterns: ['*.ipynb']
          shell_command: ipython -m ${watch_dest_path}
      
#### Run the watcher in a terminal

    watchmedo tricks tricks.yaml
      
> [`tricks.yaml`](tricks.yaml) is a concrete implementation of `tricks.yaml`

## Developer

* [Source Notebooks](src/notebooks/)
* [Transpiled Python Source](src/importnb/)
* [Tests](src/importnb/tests)

### Format and test the Source Code

In [32]:
    from IPython import get_ipython

In [33]:
    if __name__ == '__main__':
        from pathlib import Path
        from importnb.utils.export import export
        from importnb.capture import capture_output
        root = 'src/importnb/notebooks/'
        for path in Path(root).rglob("""*.ipynb"""):                
            if 'checkpoint' not in str(path):
                export(path, Path('src/importnb') / path.with_suffix('.py').relative_to(root))
        with capture_output() as out:
            !ipython -m pytest -- src 
        print('plugins'+out.stdout.split('plugins', 1)[-1])

plugins: ignore-flaky-0.1.1, forked-0.2, cov-2.5.1, benchmark-3.1.1, importnb-0.3.0
collected 24 items                                                             

src/importnb/tests/test_importnb.ipynb::test_single_file_with_context PASSED [  4%]
src/importnb/tests/test_importnb.ipynb::test_from_filename PASSED        [  8%]
src/importnb/tests/test_importnb.ipynb::test_from_execute PASSED         [ 12%]
src/importnb/tests/test_importnb.ipynb::test_with_doctest PASSED         [ 16%]
src/importnb/tests/test_importnb.ipynb::test_from_filename_main PASSED   [ 20%]
src/importnb/tests/test_importnb.ipynb::test_parameterize PASSED         [ 25%]
src/importnb/tests/test_importnb.ipynb::test_python_file PASSED          [ 29%]
src/importnb/tests/test_importnb.ipynb::test_single_file_with_capture PASSED [ 33%]
src/importnb/tests/test_importnb.ipynb::test_capturer PASSED             [ 37%]
src/importnb/tests/test_importnb.ipynb::test_single_file_with_lazy PASSED [ 41%]
src/importnb/tests/test_im

In [ ]:
    if __name__ == '__main__':
        !jupyter nbconvert --to markdown readme.ipynb

    if __name__ == '__main__':
        from IPython.display import display, Image
        !pyreverse importnb -opng -pimportnb
        display(*map(Image, ('classes_importnb.png', 'packages_importnb.png')))